In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision
!pip install underthesea

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
from underthesea import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

In [ ]:
path = "/content/drive/MyDrive/corpus/dataset.csv"
df = pd.read_csv(path)
df.head()

,title,summary,label
0,Giấc mơ hướng biển và tương lai đô thị TP HCM ...,Dự lễ khởi công Khu đô thị lấn biển Cần Giờ sa...,Thời sự
1,Nơi lưu giữ tiêm kích ném bom Tân Sơn Nhất các...,"TP HCMBảo tàng Không quân phía Nam, quận Tân B...",Thời sự
2,Hội An sẽ không còn là thành phố,Quảng NamSau gần 20 năm định danh là thành phố...,Thời sự
3,Đường Hồ Chí Minh trên biển ở Cà Mau là di tíc...,Đường Hồ Chí Minh trên biển - điểm bến Vàm Lũn...,Thời sự
4,Thủ tướng điều động công tác 4 tướng quân đội,Thủ tướng bổ nhiệm công tác mới với 4 tướng qu...,Thời sự


In [ ]:
df["text"] = df['title'] + '. ' + df['summary']
df.head()

,title,summary,label,text
0,Giấc mơ hướng biển và tương lai đô thị TP HCM ...,Dự lễ khởi công Khu đô thị lấn biển Cần Giờ sa...,Thời sự,Giấc mơ hướng biển và tương lai đô thị TP HCM ...
1,Nơi lưu giữ tiêm kích ném bom Tân Sơn Nhất các...,"TP HCMBảo tàng Không quân phía Nam, quận Tân B...",Thời sự,Nơi lưu giữ tiêm kích ném bom Tân Sơn Nhất các...
2,Hội An sẽ không còn là thành phố,Quảng NamSau gần 20 năm định danh là thành phố...,Thời sự,Hội An sẽ không còn là thành phố. Quảng NamSau...
3,Đường Hồ Chí Minh trên biển ở Cà Mau là di tíc...,Đường Hồ Chí Minh trên biển - điểm bến Vàm Lũn...,Thời sự,Đường Hồ Chí Minh trên biển ở Cà Mau là di tíc...
4,Thủ tướng điều động công tác 4 tướng quân đội,Thủ tướng bổ nhiệm công tác mới với 4 tướng qu...,Thời sự,Thủ tướng điều động công tác 4 tướng quân đội....


In [ ]:
def clean_text(text):
  if not isinstance(text, str):
    text = str(text)
  text = text.lower()
  text = re.sub(r'[^\w\s]','',text)
  text = re.sub(r'\d+','',text)
  return text

df['text'] = df['text'].apply(clean_text)
df['text'] = df['text'].apply(lambda x: word_tokenize(x, format='text'))

MÃ HÓA VĂN BẢN VÀ NHÃN


In [ ]:
#token hóa từ
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
# word_index = tokenizer.word_index
padded = pad_sequences(sequences, padding="post")
max_len = padded.shape[1]

#encode nhãn bằng số nguyên
encoder = LabelEncoder()
y = encoder.fit_transform(df["label"])

# label_tokenizer = Tokenizer()
# label_tokenizer.fit_on_texts(df['label'])
# labels = label_tokenizer.texts_to_sequences(df["label"])
# y = to_categorical([item[0] - 1 for item in labels])


XÂY DỰNG MÔ HÌNH LSTM

In [ ]:
#chia train/test
X_train, X_test, y_train, y_test = train_test_split(padded, y, test_size=0.2, random_state=42)

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

# Tính số lớp đầu ra (số nhãn)
num_classes = len(np.unique(y_train))

model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128),
    Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
from sklearn.utils import class_weight

# Tính trọng số cho từng class
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))


In [ ]:
# model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), class_weight=class_weights, callbacks=[early_stop])
model.fit(X_train, y_train, epochs=20, batch_size=64, validation_data=(X_test, y_test), class_weight=class_weights)

Epoch 1/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 150s 845ms/step - accuracy: 0.2369 - loss: 2.1553 - val_accuracy: 0.7216 - val_loss: 0.9712
Epoch 2/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 150s 901ms/step - accuracy: 0.7001 - loss: 0.9148 - val_accuracy: 0.7791 - val_loss: 0.7426
Epoch 3/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 199s 882ms/step - accuracy: 0.8410 - loss: 0.5273 - val_accuracy: 0.8101 - val_loss: 0.6549
Epoch 4/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 205s 903ms/step - accuracy: 0.8834 - loss: 0.3699 - val_accuracy: 0.8105 - val_loss: 0.6906
Epoch 5/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 200s 892ms/step - accuracy: 0.9147 - loss: 0.2841 - val_accuracy: 0.8169 - val_loss: 0.7316
Epoch 6/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 199s 874ms/step - accuracy: 0.9219 - loss: 0.2552 - val_accuracy: 0.8222 - val_loss: 0.7392
Epoch 7/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 204s 884ms/step - accuracy: 0.9336 - loss: 0.2047 - val_accuracy: 0.8200 - val_loss: 0.7730
Epoch 8/20
166/166 ━━━━━━━━━━━━━━━━━━━━ 145s 876ms/step - accuracy: 0.9476 -

LƯU MÔ HÌNH

In [ ]:
model.save('/content/drive/MyDrive/corpus/model/my_lstm_model_v2.h5')

# Lưu tokenizer
with open('/content/drive/MyDrive/corpus/model/tokenizer_v2.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Lưu label encoder (nếu bạn dùng)
with open('/content/drive/MyDrive/corpus/model/label_encoder_v2.pkl', 'wb') as f:
    pickle.dump(encoder, f)

TEST MÔ HÌNH

In [ ]:
# Load mô hình
from tensorflow.keras.models import load_model
model = load_model('/content/drive/MyDrive/corpus/model/my_lstm_model_v2.h5')

# Load tokenizer
with open('/content/drive/MyDrive/corpus/model/tokenizer_v2.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Load label encoder
with open('/content/drive/MyDrive/corpus/model/label_encoder_v2.pkl', 'rb') as f:
    encoder = pickle.load(f)

In [ ]:
# Văn bản cần phân loại
text = "Guardiola: 'Liverpool của Klopp là đối thủ lớn nhất sự nghiệp'"

# dự đoán
text = clean_text(text)
text = word_tokenize(text, format='text')
seq = tokenizer.texts_to_sequences([text])
padded = pad_sequences(seq, padding="post", maxlen=max_len)
pred = model.predict(padded)
predicted_label = encoder.inverse_transform([pred.argmax(axis=1)[0]])

print("Label dự đoán:", predicted_label[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Label dự đoán: Thể thao
